# Part 1: How NRPy+ Computes Finite Difference Coefficients

### NRPy+ Source Code for this module: [finite_difference.py](../edit/finite_difference.py)

## Introduction to Finite Differencing: Basic Definitions

Suppose we have a *uniform* numerical grid in one dimension; say, the Cartesian $x$ direction. Since the grid is uniform, the spacing between successive grid points is $\Delta x$, and the position of the $i$th point is given by

$$x_i = x_0 + i \Delta x.$$

Then, given a function $u(x)$ on this uniform grid, we will adopt the notation

$$u(x_i) = u_i.$$

We wish to approximate derivatives of $u_i$ at some nearby point (first we will consider derivatives at $x_i$ itself) using [finite difference](https://en.wikipedia.org/wiki/Finite_difference). (FD) techniques. 

FD techniques are usually equivalent to first finding the unique $N$th degree polynomial that passes through $N+1$ sampled points of our function ($u$) in the neighborhood of where we wish to find the derivative. Then, provided $u$ is smooth and properly-sampled, the $M$th derivative of the polynomial (where $M\le N-1$; *Exercise: Justify this inequality*) is an approximate $M$th derivative of $u$ called the *$M$th-order finite difference derivative*. The approximation error generally decreases as the polynomial degree or sampling density increases.

Finite difference derivatives are written in the form
$$u^{(n)}(x_i)_{\text{FD}} = \sum_{j} u_j a_j,$$
where the $a_j$'s are known as *finite difference coefficients*. For a given finite difference representation, the set of $a_j$'s are unique.

There are many ways to compute finite difference coefficients $a_j$, though perhaps the most popular involves Taylor series expansions about sampled points near the point where we wish to evaluate the derivative.

## Centered Finite Difference Representation of $u'(x_i) = u'_i$ Accurate to Fourth-order in $\Delta x$

As an illustration, let's first derive for uniform grids the centered, first-order, finite-difference coefficients accurate to fourth-order in $\Delta x$. A fourth-order-accurate centered finite-difference derivative is equivalent to the derivative of the unique polynomial fit at sampled points $\left\{j-2,j-1,j,j+1,j+2\right\}$. 

The Taylor series expansion of a function $f(x)$ about a point $x_0$ is given by

$$f(x) = \sum_{n=0}^\infty \frac{f^{(n)}(x_0)}{n!} (x-x_0)^n,$$

where $f^{(n)}(x_0)$ is the $n$th derivative of $f$ evaluated at point $x_0$. Based on this, we can immediately write the Taylor expansion of $f$ at a point $x=x_0+j\Delta x$. In this case,

\begin{align}
f(x_0+j\Delta x) &= \sum_{n=0}^\infty \frac{f^{(n)}(x_0)}{n!} (j\Delta x)^n,\text{ or equivalently:} \\
f_j &= \sum_{n=0}^\infty \frac{f^{(n)}_j}{n!} (j\Delta x)^n.
\end{align}

Thus Taylor expanding $u(x_i)$ about the sampled points $i\in \left\{j-2,j-1,j,j+1,j+2\right\}$ to $n=5$ yields the following:

\begin{align}
u_{j-2} &= u_j - (2 \Delta x) u'_j + \frac{(2 \Delta x)^2}{2} u''_j - \frac{(2 \Delta x)^3}{3!} u'''_j + \frac{(2 \Delta x)^4}{4!} u^{(4)}_j +\mathcal{O}\left((\Delta x)^5\right) \\
u_{j-1} &= u_j - (\Delta x) u'_j + \frac{(\Delta x)^2}{2} u''_j - \frac{(\Delta x)^3}{3!} u'''_j + \frac{(\Delta x)^4}{4!} u^{(4)}_j +\mathcal{O}\left((\Delta x)^5\right)\\
u_{j} &= u_j \\
u_{j+1} &= u_j + (\Delta x) u'_j + \frac{(\Delta x)^2}{2} u''_j + \frac{(\Delta x)^3}{3!} u'''_j + \frac{(\Delta x)^4}{4!} u^{(4)}_j +\mathcal{O}\left((\Delta x)^5\right)\\
u_{j+2} &= u_j + (2 \Delta x) u'_j + \frac{(2 \Delta x)^2}{2} u''_j + \frac{(2 \Delta x)^3}{3!} u'''_j + \frac{(2 \Delta x)^4}{4!} u^{(4)}_j +\mathcal{O}\left((\Delta x)^5\right)\\
\end{align}

Our goal is to compute coefficients $a_j$ such that $(a_{j-2} u_{j-2} + a_{j-1} u_{j-1}...)/(\Delta x) = u'_j + \mathcal{O}\left((\Delta x)^4\right)$. So we get

\begin{align}
& (a_{j-2} u_{j-2} + a_{j-1} u_{j-1} + a_j u_j + a_{j+1} u_{j+1} +a_{j+2} u_{j+2})/(\Delta x) \\
= & \left( u_j - (2 \Delta x) u'_j + \frac{(2 \Delta x)^2}{2} u''_j -\frac{(2 \Delta x)^3}{3!} u'''_j+\frac{(2 \Delta x)^4}{4!} u^{(4)}_j \right) a_{j-2} \\
& + \left( u_j - (\Delta x) u'_j + \frac{(\Delta x)^2}{2} u''_j - \frac{(\Delta x)^3}{3!} u'''_j+\frac{(\Delta x)^4}{4!} u^{(4)}_j \right) a_{j-1} \\
& + \left( u_j \right) a_{j} \\
& + \left( u_j + (\Delta x) u'_j + \frac{(\Delta x)^2}{2} u''_j + \frac{(\Delta x)^3}{3!} u'''_j+\frac{(\Delta x)^4}{4!} u^{(4)}_j \right) a_{j+1} \\
& + \left( u_j + (2 \Delta x) u'_j + \frac{(2 \Delta x)^2}{2} u''_j + \frac{(2 \Delta x)^3}{3!} u'''_j+\frac{(2 \Delta x)^4}{4!} u^{(4)}_j \right) a_{j+2}
\end{align}

First notice that each time we take a derivative in the Taylor
expansion, we multiply by a $\Delta x$. Notice that this helps to keep
the units consistent (e.g., if $x$ were in units of meters). Let's
just **absorb those $\Delta x$'s into the derivatives (we will extract them again later)** and
rearrange terms:

\begin{align}
& a_{j-2} u_{j-2} + a_{j-1} u_{j-1} + a_j u_j + a_{j+1} u_{j+1} + a_{j+2} u_{j+2} \\
= & \left( a_{j-2} + a_{j-1} + a_j + a_{j+1} + a_{j+2} \right) u_j \\
& + \left( -2 a_{j-2} - a_{j-1} + a_{j+1} + 2 a_{j+2} \right) u'_j \\
& + \left( 2^2 a_{j-2} + a_{j-1} + a_{j+1} + 2^2 a_{j+2} \right)/2! u''_j \\
& + \left( -2^3 a_{j-2} - a_{j-1} + a_{j+1} + 2^3 a_{j+2} \right)/3! u'''_j \\
= & u'_j
\end{align}

In order for the above to hold true for any nonzero values of
$\left\{ u_j,u'_j,u''_j,u'''_j,u^{(4)}_j\right\}$, the following set
of equations must also hold:
\begin{align}
0 &= a_{j-2} + a_{j-1} + a_j + a_{j+1} + a_{j+2}\\
1 &= -2 a_{j-2} - a_{j-1} + a_{j+1} + 2 a_{j+2}\\
0 \times 2! &= 2^2 a_{j-2} + a_{j-1} + a_{j+1} + 2^2 a_{j+2}\\
0 \times 3! &= -2^3 a_{j-2} - a_{j-1} + a_{j+1} + 2^3 a_{j+2} \\
0 \times 4! &= 2^4 a_{j-2} + a_{j-1} + a_{j+1} + 2^3 a_{j+2}.
\end{align}

Now we write this expression in matrix form (note that $0!=1$):
\begin{equation}
\left(
\begin{array}{c}
0\times 0! \\
1\times 1! \\
0\times 2! \\
0\times 3! \\
0\times 4! \\
\end{array}
\right)
=
\left(
\begin{array}{ccccc}
 1 &  1 & 1 & 1 & 1 \\
(-2)^1 &(-1)^1 & 0 & 1 & 2 \\
(-2)^2 &(-1)^2 & 0 & 1 & 2^2 \\
(-2)^3 &(-1)^3 & 0 & 1 & 2^3 \\
(-2)^4 &(-1)^4 & 0 & 1 & 2^4 \\
\end{array}
\right)
\left(
\begin{array}{c}
a_{j-2} \\
a_{j-1} \\
a_{j} \\
a_{j+1} \\
a_{j+2} \\
\end{array}
\right)
\end{equation}

So we have reduced the computation of finite difference coefficients
to the inversion of an $N\times N$ matrix equation. Notice that the
elements of the matrix will vary from the one given above if the grid
spacing is not constant, but are otherwise invariant to $\Delta x$.

The inverted matrix reads
\begin{equation}
\left(
\begin{array}{ccccc}
0 & 1/12 & -1/24 & -1/12 & 1/24 \\
0 & -2/3 & 2/3 & 1/6 & -1/6 \\
1 & 0 & -5/4 & 0 & 1/4 \\
0 & 2/3 & 2/3 & -1/6 & -1/6 \\
0 & -1/12 & -1/24 & 1/12 & 1/24 \\
\end{array}
\right)
\label{fourthorder_inv_matrix}
\end{equation}

The coefficients for the $M$th derivative can be immediately read by
multiplying the $(M+1)$st column by $M!/(\Delta x)^M$. For example, the zeroth derivative at point $i$ is given by 

$$\frac{0!}{(\Delta x)^0} \times (0 u_{i-2} + 0 u_{i-1} + u_i + 0 u_{i+1} + 0 u_{i+2}) = u_i,$$

which is exact. The first derivative finite difference approximation at point $i$ is given by

$$\frac{1!}{(\Delta x)^1} \times \left(\frac{1}{12}( u_{i-2} - u_{i+2}) + \frac{2}{3}( -u_{i-1} + u_{i+1})\right) \approx (\partial_x u)_i,$$

and the second derivative finite difference approximation at point $i$ is given by 

$$\frac{2!}{(\Delta x)^2} \times \left(-\frac{1}{24}(u_{i-2} + u_{i+2}) + \frac{2}{3}(u_{i-1} + u_{i+1}) - \frac{5}{4} u_i \right) \approx (\partial_x^2 u)_i,$$

In short, this matrix yields the finite difference derivative coefficients with the lowest possible error given a stencil size of 5 gridpoints. It can be shown by analyzing cancellations in higher order terms of the Taylor series expansions that the first and second derivative coefficients are correct to $(\Delta x)^4$ and third and fourth derivatives are correct to $(\Delta x)^2$. 

### Exercise 1: Find the exact expressions for the dominant error term on all derivatives that can be computed from this matrix (zeroth through fourth derivatives).

### Exercise 2: Construct the matrix whose inverse yields the 5-point stencil *upwinded* derivative coefficients (i.e., the stencil includes points $\{u_{i-4},u_{i-3},u_{i-2},u_{i-1},u_{i}\}$).

NRPy+ implements this simple matrix inversion strategy to evaluate finite difference coefficients.

# Digging Deeper (optional module; Part 2 below is required): Low-Level Retrieval of Finite Difference Coefficients

In Part 1 above, we learned how NRPy+ 